# Lung boundary condition for a 3d printed ventilator restrictor

Dr Daniel Duke<br>
Laboratory for Turbulence Research in Aerospace & Combustion (LTRAC)<br>
Department of Mechanical & Aerospace Engineering<br>
Monash University<br>

21st April 2020

## Compliance model
Lungs have a static compliance (typ 0.2 L/cmH2O) and a dynamic compliance with a corresponding exponential time constant of typically 0.5 s. The air in the system will also have some interial resistance which will depend on its kinematic viscosity and volume.

These can be modeled as a spring-mass-damper system. For mechanical ventilation, this is analogous to a forced spring-mass-damper system where the volume flow rate is imposed, leading to an imposed inspired volume over time, and the pressure response can then be determined as a function of time following:

$$
    \begin{align}
    m \frac{d^2p}{dt^2} + c \frac{dp}{dt} + k p &= V\\
    \text{volume} \;& V: \left[ \mathrm{L} \right]\\
    \text{pressure function} \;& p(t): \left[ \mathrm{cmH_2O} \right]\\
    \text{compliance} \;& k: \left[ \mathrm{L/cmH_2O} \right]\\
    \text{damping factor} \;& c: \left[ \mathrm{L/cmH_2O/s} \right]\\
    \text{inertial factor} \;& m: \left[ \mathrm{L/cmH_2O/s^2} \right]
    \end{align}
$$

The solution has two parts; $p(t) = p_h(t) + p_s(t)$. $p_h$ is a homogeneous solution where $\dot{V}=0$ and a $p_s$ specific solution that accounts for the volume flow. For the homogeneous part the sign of $c^2 - 4mk$ determines the form of the solution, and it will typically be positive as $m$ is quite small. 

The homogeneous solution is of the form $p_h(t) = A e^{-at} + B e^{-bt}.$ The time constants are $-a^{-1}$ and $-b^{-1}$. For simulation purposes we need an analytic solution to the function of the form $p(\dot{V},t,k,c,m)$.
    


### Dynamic Resistance

When the lung is mechanically ventilated there will also be a pressure loss in the endotracheal tube that ought to be considered, this will be a 2nd-degree function of the volume flowrate. For a normally ventilated human lung, according to Mead & Agostini:

$$
\begin{align}
    \Delta p &= 0.24 \dot{V} + 0.03 \dot{V}^2\\
    \text{volume flow rate} \;& \dot{V}: \left[ \mathrm{L \cdot min}^{-1} \right]\\
    \text{pressure gradient} \;& \Delta p: \left[ \mathrm{kPa} \right]
\end{align}
$$

### Original boundary condition

outlet_ur
    {
        /* 
         - This is a lung compliance simulation function.
         The concept is that the stagnation pressure at the patch rises proportionally to the volume flow rate
         through the patch. Compliance is set in Litres per cm of H2O and converted to SI units in the function.
         Tidal volume is in Litres but initial pressure is in Pascals.
         - Static pressure is set from a calculated stagnation pressure (gauge rel to 100 kPa)
         - Stagnation pressure at lung is set as integrated volume flow divided by compliance
         - integrated volume flow is determined as the time average volume flux multiplied by elapsed time
           Since flux is both patch-area and time-integrated, it should be stable against fluctuations.
         - Once the total volume flowed passes the tidal volume, the compliance becomes very small and the pressure
         will rise very quickly with further flow until such time as the pressure gradient vanishes and flow stops.

           Current settings:
                - 0 cmH2O gauge = 100 kpa abs.
                - Starting lung pressure is 100294.20 Pa to match inlet profile from experiment
                - Compliance of 0.1 L/cmH2O on the unrestricted port (a poor lung)
                - Compliance of 0.2 L/cmH2O on the restricted port (a healthy lung)
                - Max tidal volume of 0.5 L
                - Maximum allowed pressure of +41 cmH2O
        */

        type    groovyBC;
        valueExpression "(lung_p0g1 > 104e3 ? 104e3 : lung_p0g1) - 0.5*rho*U&U";
        variables (
                    "compliance1=0.1;"
                    "initial_pressure1=100294.20;"
                    "tidalVolume1=0.5;"
                    "integratedVolumetricFlow1=time()*sum(mag(UMean)*mag(Sf()));"
                    "limit1=((mag(integratedVolumetricFlow1)>tidalVolume1*1e-3) ? exp(10*(1 - mag(integratedVolumetricFlow1)*1e3/tidalVolume1)) : 1);"
                    "lung_p0g1=initial_pressure1+integratedVolumetricFlow1/(compliance1*limit1*1.01972e-5);"
        );

        value uniform 100294.20;
    }